# Import Event Log

In [37]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/BPIC12_Log_onlyO.csv', sep=',')

    # Drop the first column without knowing its name
    dataframe_log = dataframe_log.drop(dataframe_log.columns[0], axis=1)

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_4946/670060893.py:14: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


# Drop unnessary columns

In [38]:
dataframe_log = dataframe_log.drop(columns=['lifecycle:transition'])
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])
dataframe_log = dataframe_log.drop(columns=['@@index'])
dataframe_log = dataframe_log.drop(columns=['case:AMOUNT_REQ'])
dataframe_log = dataframe_log.drop(columns=['case:REG_DATE'])

In [39]:
dataframe_log

,org:resource,concept:name,time:timestamp,@@case_index
0,10862,O_SELECTED,2011-10-01 09:45:09.243000+00:00,0
1,10862,O_CREATED,2011-10-01 09:45:11.197000+00:00,0
2,10862,O_SENT,2011-10-01 09:45:11.380000+00:00,0
3,11049,O_SENT_BACK,2011-10-10 09:33:03.668000+00:00,0
4,10629,O_ACCEPTED,2011-10-13 08:37:29.226000+00:00,0
...,...,...,...,...
31239,11003,O_SENT,2012-03-02 09:14:43.303000+00:00,5013
31240,10789,O_SENT_BACK,2012-03-09 10:01:46.008000+00:00,5013
31241,10933,O_SELECTED,2012-03-01 19:22:38.593000+00:00,5014
31242,10933,O_CREATED,2012-03-01 19:22:40.016000+00:00,5014


# Preprocess

In [40]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [41]:
codes, uniques = pd.factorize(dataframe_log['org:resource'])
dataframe_log['org:resource'] = codes + 1

In [42]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

In [43]:
dataframe_log

,org:resource,concept:name,@@case_index,standardized_elapsed_time
0,1,1,0,-0.620650
1,1,2,0,-0.620647
2,1,3,0,-0.620647
3,2,4,0,0.299272
4,3,5,0,0.602251
...,...,...,...,...
31239,52,3,5013,-0.620265
31240,26,4,5013,0.099241
31241,46,1,5014,-0.620650
31242,46,2,5014,-0.620648


# Generate Prefixes

In [44]:
df_resource = dataframe_log[['org:resource', '@@case_index']]
df_activity = dataframe_log[['concept:name', '@@case_index']]
df_timestamp = dataframe_log[['standardized_elapsed_time', '@@case_index']]

In [45]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array

In [46]:
windows_resource, targets_resource, case_indices = generate_prefix_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)
windows_timestamp, targets_timestamp, case_indices = generate_prefix_windows(df_timestamp)

KeyboardInterrupt: 

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [ ]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_activities, num_resources, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_activity = Input(shape=(None,), name='activity_input')
    input_resource = Input(shape=(None,), name='resource_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')

    # Embedding layers for categorical attributes
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_resource = Embedding(input_dim=num_resources + 1, output_dim=embedding_dim, mask_zero=True, name='resource_embedding')(input_resource)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_resource = GRU(units=gru_units, return_sequences=True, name='resource_encoder')(embedding_resource)
    bn_resource = BatchNormalization(name='bn_resource')(encoded_resource)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_activity, bn_resource, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_resource = Dense(num_resources + 1, activation='softmax', name='output_resource')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)

    # Building the model
    model = Model(inputs=[input_activity, input_resource, input_timestamp], outputs=[output_activity, output_resource, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_activity': 'categorical_crossentropy', 
            'output_resource': 'categorical_crossentropy',
            'output_timestamp': 'mse'
        },
        metrics={
            'output_activity': ['accuracy'], 
            'output_resource': ['accuracy'],
            'output_timestamp': ['mse']
        }
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_activities = dataframe_log['concept:name'].max()
num_resources = dataframe_log['org:resource'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_activities, num_resources, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 resource_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 activity_embedding (Embedd  (None, None, 50)             400       ['activity_input[0][0]']      
 ing)                                                                                         

### Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data for the resource attribute
train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

# Split the data for the activity attribute
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Split the data for the timestamp attribute
train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)

### Training

In [ ]:
from tensorflow.keras.utils import to_categorical

train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources + 1)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources + 1)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

In [ ]:
history = model.fit(
    [train_activity, train_resource, train_timestamp],
    [train_targets_activity_cat, train_targets_resource_cat, train_targets_timestamp],
    validation_data=([test_activity, test_resource, test_timestamp], [test_targets_activity_cat, test_targets_resource_cat, test_targets_timestamp]),
    epochs=20,
    batch_size=500
)

Epoch 1/20
37/37 [==============================] - 32s 442ms/step - loss: 5.4616 - output_activity_loss: 1.2929 - output_resource_loss: 3.7198 - output_timestamp_loss: 0.4489 - output_activity_accuracy: 0.5641 - output_resource_accuracy: 0.1586 - output_timestamp_mse: 0.4489 - val_loss: 6.9391 - val_output_activity_loss: 1.8967 - val_output_resource_loss: 4.0243 - val_output_timestamp_loss: 1.0180 - val_output_activity_accuracy: 0.5797 - val_output_resource_accuracy: 0.1941 - val_output_timestamp_mse: 1.0180
Epoch 2/20
37/37 [==============================] - 24s 659ms/step - loss: 3.7857 - output_activity_loss: 0.6620 - output_resource_loss: 2.7419 - output_timestamp_loss: 0.3818 - output_activity_accuracy: 0.7764 - output_resource_accuracy: 0.4672 - output_timestamp_mse: 0.3818 - val_loss: 6.6234 - val_output_activity_loss: 1.7744 - val_output_resource_loss: 3.8601 - val_output_timestamp_loss: 0.9889 - val_output_activity_accuracy: 0.6639 - val_output_resource_accuracy: 0.4519 - val

In [ ]:
# Evaluate the model on the validation set
results = model.evaluate(
    [test_activity, test_resource, test_timestamp],
    [test_targets_activity_cat, test_targets_resource_cat, test_targets_timestamp],
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

123/123 [==============================] - 2s 17ms/step - loss: 1.7534 - output_activity_loss: 0.4051 - output_resource_loss: 1.0790 - output_timestamp_loss: 0.2693 - output_activity_accuracy: 0.8206 - output_resource_accuracy: 0.6963 - output_timestamp_mse: 0.2693
Validation Loss: 1.7534207105636597, Validation Accuracy: 0.40506815910339355


In [ ]:
# Save the model to an H5 file
model.save('binetv3_BPIC_O.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [ ]:
# Generate predictions for all inputs
predictions = model.predict([windows_activity, windows_resource, windows_timestamp])


# Extract predictions for categorical attributes (softmax probabilities)
predictions_activity = predictions[0]
predictions_resource = predictions[1]
predictions_timestamp = predictions[2]

820/820 [==============================] - 13s 10ms/step


In [ ]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [ ]:
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))
anomaly_scores_resource = calculate_anomaly_scores(predictions_resource, targets_resource.astype(int))

In [ ]:
def compute_anomaly_scores_continuous(predictions, actuals):
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize to [0, 1] range
    max_diff = np.max(differences)
    normalized_scores = differences / max_diff if max_diff != 0 else differences
    
    return normalized_scores

In [ ]:
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp)
anomaly_scores_timestamp = anomaly_scores_timestamp.flatten()
anomaly_scores_timestamp = anomaly_scores_timestamp.tolist()

## Insert missing scores for cases with less than 2 Events

In [ ]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_resource'] = anomaly_scores_resource
score['score_activity'] = anomaly_scores_activity
score['score_timestamp'] = anomaly_scores_timestamp

score

,case,score_resource,score_activity,score_timestamp
0,0,0.000000,0.0,0.002552
1,0,0.000000,0.0,0.005113
2,0,0.141140,0.0,0.015123
3,0,0.473138,0.0,0.010280
4,1,0.000000,0.0,0.002552
...,...,...,...,...
26224,5013,0.000000,0.0,0.009595
26225,5013,0.000000,0.0,0.000124
26226,5013,0.793720,0.0,0.064231
26227,5014,0.000000,0.0,0.003741


In [ ]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(0, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 5014

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")

Does the 'case' column contain all values between 0 and 5014? True


### Threshold (lowest plateau)

In [ ]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [ ]:
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_resource = find_plateaus(anomaly_scores_resource)
threshold_timestamp = find_plateaus(anomaly_scores_timestamp)

### Detection

In [ ]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [ ]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_resource = detect_anomalies(anomaly_scores_resource, threshold_resource)
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_timestamp = detect_anomalies(anomaly_scores_timestamp, threshold_timestamp)

# Mapping

In [ ]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_resource'] = labels_resource
mapping['predicted_activity'] = labels_activity
mapping['predicted_timestamp'] = labels_timestamp

mapping

,case,predicted_resource,predicted_activity,predicted_timestamp
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,1,0,0,0
...,...,...,...,...
26224,5013,0,0,0
26225,5013,0,0,0
26226,5013,1,0,0
26227,5014,0,0,0


In [ ]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_resource', 'predicted_activity', 'predicted_timestamp']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0       False
1        True
2        True
3        True
4        True
        ...  
5010    False
5011    False
5012     True
5013     True
5014    False
Length: 5015, dtype: bool

# Ground Truth

In [ ]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [ ]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_O.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/168 [00:00<?, ?it/s]

In [ ]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [47]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]

# Evaluation

In [48]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [49]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.70


In [50]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.91


In [51]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.81


In [52]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.51


In [54]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.71
